In [ ]:
import requests
import re

# Azure OpenAI API 엔드포인트 및 API 키
endpoint = "https://6a005-openai-eastus2.openai.azure.com/openai/deployments/6a005-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"
api_key = "4n4wU9MTtdMeHXhBX1fAURyZKJTWhaweFWxSO4GLqomzHbV4yKOVJQQJ99BCACLArgHXJ3w3AAABACOG8d7A"

# Azure Search 관련 설정
ai_search_endpoint = "https://6a005-ai-search.search.windows.net"
ai_search_api_key = "KLsyBFpDEWPWczaOTmgDOvxuvbrwPGv8N6WZ9yK6PZAzSeCIsFzo"
ai_search_index = "venture-index-microsoft"

def request_gpt(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    
    body = {
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": ai_search_endpoint,
                    "index_name": ai_search_index,
                    "semantic_configuration": "venture-semantic",
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "in_scope": True,
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": ai_search_api_key
                    },
                    "key": ai_search_api_key,
                }
            }
        ],
        "messages": [
            {
                "role": "system",
                "content": "너는 중소기업 경영자에게 지원사업 정보를 제공하는 도우미야"
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }
    
    response = requests.post(endpoint, headers=headers, json=body)
    
    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        # context가 없는 경우도 있을 수 있으므로 안전하게 가져옴
        citation_list = message.get('context', {}).get('citations', [])
        content = message['content']
        # [doc숫자]를 [참조 숫자]로 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content, citation_list
    else:
        print("Error:", response.status_code, response.text)
        return "응답 에러 발생", []

assistant


부산에서 고령친화 인력 사업을 운영하시면서 3월에 참여할 수 있는 지원사업에 대한 정보는 다음과 같습니다:

1. **2024년 고령친화산업 재정지원사업**
   - **신청 종료일자**: 2024년 3월 31일
   - **공고 URL**: [부산광역시](https://www.busan.go.kr)

2. **2024년 고령친화 인력 양성 프로그램**
   - **신청 종료일자**: 2024년 3월 15일
   - **공고 URL**: [부산광역시](https://www.busan.go.kr)

3. **2024년 고령친화 기업 지원사업**
   - **신청 종료일자**: 2024년 3월 20일
   - **공고 URL**: [부산광역시](https://www.busan.go.kr)

4. **2024년 고령친화 서비스 개발 지원**
   - **신청 종료일자**: 2024년 3월 10일
   - **공고 URL**: [부산광역시](https://www.busan.go.kr)

5. **2024년 고령친화 사회적 기업 지원사업**
   - **신청 종료일자**: 2024년 3월 25일
   - **공고 URL**: [부산광역시](https://www.busan.go.kr)

각 지원사업의 세부사항은 부산광역시의 공식 웹사이트를 통해 확인하실 수 있습니다. 필요하신 추가 정보가 있으시면 말씀해 주세요!

참조 1
부산테크노파크
[부산] 2024년 고령친화산업 육성지원사업 지원기업 모집 공고
기술
부산광역시
참조 2
국가과학기술연구회
2024년 3월 공공연 연구인력 파견지원 공고(중소기업 연구인력지원사업)
인력
중소벤처기업부
참조 3
한국고용정보원
2024년 1분기 (최초) 고령자 고용지원금 신청서 접수 공고
인력
고용노동부
참조 4
부산광역여성새로일하기센터
[부산] 2024년 여성 창업자 맞춤형 전문가 컨설팅 신청 공고
창업
부산광역시
참조 5
부산경제진흥원
[부산] 2024년 부산창업카페 전문가 컨설팅 지원 공고
창업
부산광역시


In [14]:
custom_css = """
:root {
  --primary-color: #007FFF;       /* Azure Blue */
  --secondary-color: #005BB5;     /* 약간 더 어두운 블루 */
  --background-color: #F0F8FF;      /* 연한 블루 배경 (AliceBlue) */
  --text-color: #333333;
}

/* 버튼 스타일 */
button {
  background-color: var(--primary-color);
  border: 1px solid var(--secondary-color);
  color: white;
}

/* 텍스트 및 기타 컴포넌트 색상 */
.gradio-container {
  background-color: var(--background-color);
  color: var(--text-color);
}

/* 마크다운 제목 스타일 */
.markdown h1, .markdown h2, .markdown h3 {
  color: var(--primary-color);
}
"""

In [ ]:
import gradio as gr

def click_send(prompt, history):
    # 사용자의 프롬프트를 받아 GPT API에 요청
    content, citation_list = request_gpt(prompt)

    # 대화 기록에 사용자와 어시스턴트의 메시지를 추가 (각 메시지를 튜플로 구성)
    history.append(("사용자", prompt))
    history.append(("GPT", content))

    # 참조 목록을 문자열로 구성
    citations_text = ""
    for i, citation in enumerate(citation_list, start=1):
        citations_text += f"참조 {i}: {citation.get('content', '')}\n\n"

    # 반환: 업데이트된 대화 기록, 참조 텍스트, 프롬프트 입력창 초기화를 위한 빈 문자열
    return history, citations_text, ""

with gr.Blocks(css=custom_css) as demo:
    
    gr.HTML(
        """
        <div style="display: flex; justify-content: space-between; align-items: center;">
            <h1>중소벤처 지원사업 도우미</h1>
            <img src="https://raw.githubusercontent.com/ToleranceKim/azure-open-ai/refs/heads/main/gradio/logo/1.%EC%A4%91%EC%86%8C%EB%B2%A4%EC%B2%98%EA%B8%B0%EC%97%85%EB%B6%80_%EA%B5%AD_%EA%B0%80%EB%A1%9C.jpg" 
                 alt="중소벤처기업부 로고"
                 style="height: 50px;">
        </div>
        """
    )

    # 대화 기록을 보여줄 챗봇 컴포넌트
    chatbot = gr.Chatbot(label="나의 GPT", height=300)

    # 프롬프트 입력창과 전송 버튼
    with gr.Row():
        prompt_textbox = gr.Textbox(label="프롬프트", placeholder="프롬프트를 입력하세요.", scale=6)
        send_button = gr.Button("전송", scale=1)

    # 참조 내용을 출력할 텍스트박스
    citations_textbox = gr.Textbox(label="참조", lines=5)

    # 전송 버튼 클릭 시, click_send 함수 호출:
    # - inputs: 사용자가 입력한 프롬프트와 현재의 대화 기록(챗봇 내용)
    # - outputs: 업데이트된 대화 기록, 참조 텍스트, 그리고 프롬프트 입력창 초기화
    send_button.click(click_send, inputs=[prompt_textbox, chatbot], outputs=[chatbot, citations_textbox, prompt_textbox])

demo.launch()

c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.
